# Import all zillow rent indices

In [4]:
import pandas as pd

#filepath input is for if directories are different on each machine


#if csvs are in same directory as notebook, set filepath = './'
filepath = input('Enter file path where csv files are located on your machine\n')

# my filepath is ../../renycdsa7parkcapstonekickoff

multi = pd.read_csv(filepath + '/Zip_Zri_MultiFamilyResidenceRental.csv',dtype = {'RegionName':str})
# homes = pd.read_csv(filepath + '/Zip_Zri_AllHomesPlusMultifamily.csv',dtype = {'RegionName':str})
# single = pd.read_csv(filepath + '/Zip_Zri_SingleFamilyResidenceRental.csv',dtype = {'RegionName':str})

def zero_zips(df):
    '''Adds zeros to beginning of zip codes that were chopped off when of int dtype
    
    Args:
    
    df: name of the DataFrame'''
    return df['RegionName'].apply(lambda Z: '0'*(5-len(Z))+Z)

multi['RegionName'] = zero_zips(multi)
# homes['RegionName'] = zero_zips(homes)
# single['RegionName'] = zero_zips(single)

print(multi.shape)
# print(single.shape)
# print(homes.shape)

Enter file path where csv files are located on your machine
../../renycdsa7parkcapstonekickoff
(1861, 120)
(15836, 111)
(13181, 120)


In [5]:
multi = pd.melt(multi, id_vars =multi.columns[:7],
                value_vars = multi.columns[7:],
               var_name = 'Date',value_name = 'Rent')
multi['Date'] = pd.to_datetime(multi['Date'])
multi.rename(columns = {'RegionName':'Zipcode'}, inplace = True)



multi.sample(10)

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,Date,Rent
183316,93244,80112,Centennial,CO,Denver-Aurora-Lakewood,Arapahoe County,939,2018-11-01,1774.0
194808,59444,04330,Augusta,ME,Augusta-Waterville,Kennebec County,1265,2019-05-01,957.0
31040,59444,04330,Augusta,ME,Augusta-Waterville,Kennebec County,1265,2012-01-01,NaN
142574,58871,02720,Fall River,MA,Providence-Warwick,Bristol County,1139,2017-01-01,NaN
129286,93304,80228,Lakewood,CO,Denver-Aurora-Lakewood,Jefferson County,878,2016-06-01,NaN
177570,66845,21236,Nottingham,MD,Baltimore-Columbia-Towson,Baltimore County,776,2018-08-01,NaN
11004,58760,02451,Waltham,MA,Boston-Cambridge-Newton,Middlesex County,1700,2011-02-01,NaN
157537,95301,87106,Albuquerque,NM,Albuquerque,Bernalillo County,1214,2017-09-01,746.0
148552,81286,53206,Milwaukee,WI,Milwaukee-Waukesha-West Allis,Milwaukee County,1534,2017-04-01,679.0
73178,94852,85260,Scottsdale,AZ,Phoenix-Mesa-Scottsdale,Maricopa County,600,2013-12-01,NaN


## Select Austin, TX data

In [221]:
austin = multi.loc[(multi['City'] == 'Austin') & (multi['State']=='TX')].set_index('Date')

austin.head()

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,Rent
Date,,,,,,,,
2010-09-01,92617,78704,Austin,TX,Austin-Round Rock,Travis County,65,1349.0
2010-09-01,92654,78745,Austin,TX,Austin-Round Rock,Travis County,83,1340.0
2010-09-01,92657,78748,Austin,TX,Austin-Round Rock,Travis County,282,NaN
2010-09-01,92667,78758,Austin,TX,Austin-Round Rock,Travis County,287,907.0
2010-09-01,92651,78741,Austin,TX,Austin-Round Rock,Travis County,304,1222.0


## Impute for each zip code

In [222]:
fill_rents = austin[['Zipcode','Rent']].copy()
austin.drop('Rent', axis = 1, inplace = True)
fill_rents = fill_rents.reset_index().pivot(index = 'Date',columns = 'Zipcode')['Rent'].reset_index()
# for code in fill_rents.columns[1:]:
#     fill_rents[code].interpolate(inplace = True)
    
fill_rents.fillna(method = 'bfill',inplace = True)

fill_rents.isnull().sum()

fill_rents

Zipcode,Date,78701,78702,78703,78704,78705,78717,78723,78727,78728,...,78748,78749,78750,78751,78752,78753,78754,78757,78758,78759
0,2010-09-01,1524.0,1392.0,1377.0,1349.0,1129.0,1361.0,923.0,1350.0,975.0,...,949.0,1022.0,1012.0,1060.0,960.0,834.0,879.0,1101.0,907.0,1168.0
1,2010-10-01,1533.0,1392.0,1368.0,1342.0,1135.0,1361.0,855.0,1350.0,975.0,...,949.0,1022.0,993.0,1044.0,960.0,782.0,879.0,1101.0,885.0,1168.0
2,2010-11-01,1562.0,1392.0,1382.0,1341.0,1138.0,1361.0,855.0,1350.0,975.0,...,949.0,1022.0,974.0,1031.0,960.0,782.0,879.0,1101.0,861.0,1168.0
3,2010-12-01,1595.0,1392.0,1400.0,1354.0,1106.0,1361.0,840.0,1350.0,975.0,...,949.0,1022.0,957.0,1032.0,960.0,768.0,879.0,1101.0,839.0,1168.0
4,2011-01-01,1616.0,1392.0,1384.0,1195.0,1057.0,1361.0,830.0,1350.0,975.0,...,949.0,1022.0,935.0,1031.0,960.0,767.0,879.0,1101.0,826.0,1168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2019-09-01,2309.0,1694.0,1925.0,1586.0,1607.0,1406.0,1499.0,1380.0,1232.0,...,1425.0,1389.0,1342.0,1417.0,1477.0,1216.0,1284.0,1320.0,1392.0,1625.0
109,2019-10-01,2327.0,1732.0,1946.0,1619.0,1608.0,1441.0,1483.0,1392.0,1251.0,...,1394.0,1409.0,1352.0,1411.0,1452.0,1227.0,1285.0,1331.0,1372.0,1586.0
110,2019-11-01,2336.0,1750.0,1970.0,1610.0,1611.0,1452.0,1488.0,1393.0,1269.0,...,1416.0,1399.0,1375.0,1423.0,1471.0,1251.0,1303.0,1361.0,1382.0,1586.0
111,2019-12-01,2326.0,1735.0,1943.0,1592.0,1613.0,1453.0,1497.0,1392.0,1275.0,...,1358.0,1409.0,1387.0,1419.0,1489.0,1273.0,1310.0,1363.0,1398.0,1613.0


In [223]:
fill_rents = pd.melt(fill_rents, id_vars='Date', value_vars = fill_rents.columns[1:],var_name='Zipcode',value_name = 'Rent')

fill_rents.head()

,Date,Zipcode,Rent
0,2010-09-01,78701,1524.0
1,2010-10-01,78701,1533.0
2,2010-11-01,78701,1562.0
3,2010-12-01,78701,1595.0
4,2011-01-01,78701,1616.0


In [224]:
austin = pd.merge(austin, fill_rents, on = ['Date','Zipcode'])

In [225]:
austin.head()

,Date,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,Rent
0,2010-09-01,92617,78704,Austin,TX,Austin-Round Rock,Travis County,65,1349.0
1,2010-09-01,92654,78745,Austin,TX,Austin-Round Rock,Travis County,83,1340.0
2,2010-09-01,92657,78748,Austin,TX,Austin-Round Rock,Travis County,282,949.0
3,2010-09-01,92667,78758,Austin,TX,Austin-Round Rock,Travis County,287,907.0
4,2010-09-01,92651,78741,Austin,TX,Austin-Round Rock,Travis County,304,1222.0


# Importing data from BLS and BEA

In [94]:
# labor = pd.read_excel('~/Downloads/atx_laborforce.xlsx',skiprows=10)


# labor = pd.melt(labor, id_vars = 'Year', value_vars = labor.columns[1:],var_name = 'Month', value_name = 'LaborForce')

# labor.sample(15)

In [34]:
personal = pd.read_csv('~/Downloads/atx_personal.csv',skiprows = 4)

personal.head()

,GeoFips,GeoName,LineCode,Description,2013,2014,2015,2016,2017,2018
0,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",1.0,Personal income (thousands of dollars),88954305.0,97179427.0,103077251.0,108054841.0,118651329.0,129146253.0
1,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",2.0,Population (persons) 1/,1883315.0,1942467.0,2002134.0,2062016.0,2115475.0,2165497.0
2,12420,"Austin-Round Rock-Georgetown, TX (Metropolitan...",3.0,Per capita personal income (dollars) 2/,47233.0,50029.0,51484.0,52403.0,56087.0,59638.0
3,Legend / Footnotes:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/ Census Bureau midyear population estimates....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
leisure = pd.read_excel('~/Downloads/atx_leisure.xlsx',skiprows = 12)
leisure.head(12)

leisure = pd.melt(leisure, id_vars = 'Year', value_vars = leisure.columns[1:],var_name = 'Month', value_name = 'NumEmp')
leisure['NumEmp'] = leisure['NumEmp'].apply(lambda N: N*1000)
leisure.sample(15)

atx = pd.merge(labor,leisure, on = ['Year','Month'])
atx.sample(15)

atx.to_csv('atx_leisure_labor.csv',index=False)

In [35]:
personal= personal.loc[:2,:]
personal = pd.melt(personal, id_vars = 'Description', value_vars = personal.columns[4:],var_name='Year')

personal.head()

,Description,Year,value
0,Personal income (thousands of dollars),2013,88954305.0
1,Population (persons) 1/,2013,1883315.0
2,Per capita personal income (dollars) 2/,2013,47233.0
3,Personal income (thousands of dollars),2014,97179427.0
4,Population (persons) 1/,2014,1942467.0


In [36]:
personal = personal.pivot(index = 'Year',columns = 'Description')['value'].reset_index()

personal.columns = ['Year','Income_per_capita','Personal_income','Population']

pers_inc = personal[['Year','Income_per_capita','Population']].copy()

In [226]:
#made the csv in another directory
atx = pd.read_csv('../../renycdsa7parkcapstonekickoff/atx_leisure_labor.csv')

In [227]:
pers_inc['Year'] = pers_inc['Year'].astype(int)
atx = pd.merge(atx,pers_inc, how = 'left', on = 'Year')

atx.sample(15)

,Year,Month,LaborForce,NumEmp,Income_per_capita,Population
35,2013,Apr,1015041.0,100100.0,47233.0,1883315.0
23,2012,Mar,978978.0,92200.0,NaN,NaN
51,2018,May,1195229.0,132300.0,59638.0,2165497.0
64,2020,Jun,1215025.0,99300.0,NaN,NaN
93,2016,Sep,1127416.0,124000.0,52403.0,2062016.0
7,2018,Jan,1178562.0,124100.0,59638.0,2165497.0
22,2011,Mar,952924.0,88600.0,NaN,NaN
122,2012,Dec,1002030.0,97200.0,NaN,NaN
90,2013,Sep,1026197.0,102300.0,47233.0,1883315.0
2,2013,Jan,1005093.0,94500.0,47233.0,1883315.0


In [228]:
import calendar

month_num = {month: index for index, month in enumerate(calendar.month_abbr) if month}

month_num

{'Jan': 1,
 'Feb': 2,
 'Mar': 3,
 'Apr': 4,
 'May': 5,
 'Jun': 6,
 'Jul': 7,
 'Aug': 8,
 'Sep': 9,
 'Oct': 10,
 'Nov': 11,
 'Dec': 12}

In [229]:
atx['Month'] = atx['Month'].apply(lambda M: month_num[M])

atx.sample()

,Year,Month,LaborForce,NumEmp,Income_per_capita,Population
35,2013,4,1015041.0,100100.0,47233.0,1883315.0


In [230]:
atx['Date'] = atx[['Year','Month']].apply(lambda D: '-'.join(D.values.astype(str)), axis = 1)
atx['Date'] = pd.to_datetime(atx['Date'])

In [64]:
atx.head()

,Year,Month,LaborForce,NumEmp,Income_per_capita,Population,Date
0,2011,1,944609.0,84800.0,NaN,NaN,2011-01-01
1,2012,1,973023.0,88800.0,NaN,NaN,2012-01-01
2,2013,1,1005093.0,94500.0,47233.0,1883315.0,2013-01-01
3,2014,1,1035195.0,100300.0,50029.0,1942467.0,2014-01-01
4,2015,1,1055960.0,106800.0,51484.0,2002134.0,2015-01-01


## Import credit card data from CFPB

In [231]:
cc_volume = pd.read_csv('~/Downloads/volume_data_Income_Level_CRC.csv')

In [232]:
#keep only the low and moderate income data

cc_volume = cc_volume.loc[cc_volume['income_level_group'].isin(['Low','Moderate'])]

cc_volume.head(10)

,month,date,vol,vol_unadj,income_level_group
2,60,2005-01,4.578106e+09,3.978007e+09,Moderate
3,60,2005-01,7.635135e+08,6.517910e+08,Low
6,61,2005-02,4.563908e+09,3.900504e+09,Moderate
7,61,2005-02,7.558710e+08,6.549358e+08,Low
10,62,2005-03,4.762458e+09,4.775794e+09,Moderate
11,62,2005-03,8.171551e+08,8.352900e+08,Low
14,63,2005-04,4.744005e+09,4.683566e+09,Moderate
15,63,2005-04,7.854169e+08,7.721479e+08,Low
18,64,2005-05,4.752915e+09,4.608128e+09,Moderate
19,64,2005-05,8.289172e+08,8.354502e+08,Low


In [233]:
cc_volume = cc_volume.pivot(index = 'date', columns = 'income_level_group')['vol'].reset_index()

cc_volume.columns = ['Date','Low_income_volume','Moderate_income_volume']

cc_volume.head()

,Date,Low_income_volume,Moderate_income_volume
0,2005-01,7.635135e+08,4.578106e+09
1,2005-02,7.558710e+08,4.563908e+09
2,2005-03,8.171551e+08,4.762458e+09
3,2005-04,7.854169e+08,4.744005e+09
4,2005-05,8.289172e+08,4.752915e+09


In [234]:
cc_volume['Date'] = pd.to_datetime(cc_volume['Date'])

In [235]:
cc_volume

,Date,Low_income_volume,Moderate_income_volume
0,2005-01-01,7.635135e+08,4.578106e+09
1,2005-02-01,7.558710e+08,4.563908e+09
2,2005-03-01,8.171551e+08,4.762458e+09
3,2005-04-01,7.854169e+08,4.744005e+09
4,2005-05-01,8.289172e+08,4.752915e+09
...,...,...,...
158,2018-03-01,8.935432e+08,4.476840e+09
159,2018-04-01,8.753101e+08,4.440339e+09
160,2018-05-01,8.904970e+08,4.571036e+09
161,2018-06-01,8.982546e+08,4.398129e+09


In [236]:
austin = pd.merge(austin,cc_volume, how = 'left', on = 'Date')

In [237]:
austin = pd.merge(austin, atx, how = 'left', on = 'Date')

In [238]:
austin.sample(15)

,Date,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,Rent,Low_income_volume,Moderate_income_volume,Year,Month,LaborForce,NumEmp,Income_per_capita,Population
895,2013-06-01,92651,78741,Austin,TX,Austin-Round Rock,Travis County,304,1000.0,5.846078e+08,3.014526e+09,2013.0,6.0,1023916.0,103100.0,47233.0,1883315.0
2464,2018-04-01,92653,78744,Austin,TX,Austin-Round Rock,Travis County,752,1296.0,8.753101e+08,4.440339e+09,2018.0,4.0,1195861.0,130500.0,59638.0,2165497.0
2936,2019-09-01,92666,78757,Austin,TX,Austin-Round Rock,Travis County,1380,1320.0,NaN,NaN,2019.0,9.0,1244523.0,135100.0,NaN,NaN
1264,2014-07-01,92663,78754,Austin,TX,Austin-Round Rock,Travis County,1542,928.0,7.296026e+08,4.050310e+09,2014.0,7.0,1052711.0,108600.0,50029.0,1942467.0
2375,2017-12-01,92641,78730,Austin,TX,Austin-Round Rock,Travis County,1831,1243.0,9.037584e+08,4.434000e+09,2017.0,12.0,1171877.0,126900.0,56087.0,2115475.0
1521,2015-05-01,92658,78749,Austin,TX,Austin-Round Rock,Travis County,919,1143.0,8.033060e+08,3.994612e+09,2015.0,5.0,1073073.0,117000.0,51484.0,2002134.0
1176,2014-04-01,92639,78728,Austin,TX,Austin-Round Rock,Travis County,1246,983.0,7.321531e+08,3.753325e+09,2014.0,4.0,1044885.0,106400.0,50029.0,1942467.0
743,2012-12-01,92618,78705,Austin,TX,Austin-Round Rock,Travis County,1223,1266.0,5.099254e+08,2.893411e+09,2012.0,12.0,1002030.0,97200.0,NaN,NaN
2760,2019-03-01,92668,78759,Austin,TX,Austin-Round Rock,Travis County,358,1553.0,NaN,NaN,2019.0,3.0,1228466.0,134000.0,NaN,NaN
1875,2016-06-01,92638,78727,Austin,TX,Austin-Round Rock,Travis County,1090,1350.0,9.758423e+08,4.751104e+09,2016.0,6.0,1118942.0,124400.0,52403.0,2062016.0


In [274]:
austin_df = austin.drop(['RegionID','City','State','Metro','CountyName','Year','Month','NumEmp'],axis = 1).\
sort_values(by = 'Date').set_index('Date')

In [275]:
austin_df.head(15)

,Zipcode,SizeRank,Rent,Low_income_volume,Moderate_income_volume,LaborForce,Income_per_capita,Population
Date,,,,,,,,
2010-09-01,78704,65,1349.0,3.727692e+08,2.150781e+09,NaN,NaN,NaN
2010-09-01,78730,1831,1243.0,3.727692e+08,2.150781e+09,NaN,NaN,NaN
2010-09-01,78701,1749,1524.0,3.727692e+08,2.150781e+09,NaN,NaN,NaN
2010-09-01,78752,1593,960.0,3.727692e+08,2.150781e+09,NaN,NaN,NaN
2010-09-01,78751,1558,1060.0,3.727692e+08,2.150781e+09,NaN,NaN,NaN
2010-09-01,78754,1542,879.0,3.727692e+08,2.150781e+09,NaN,NaN,NaN
2010-09-01,78735,1518,1375.0,3.727692e+08,2.150781e+09,NaN,NaN,NaN
2010-09-01,78757,1380,1101.0,3.727692e+08,2.150781e+09,NaN,NaN,NaN
2010-09-01,78750,1325,1012.0,3.727692e+08,2.150781e+09,NaN,NaN,NaN


In [276]:
austin_df = austin_df['2013-01-01':'2018-07-01']

In [277]:
austin_df

,Zipcode,SizeRank,Rent,Low_income_volume,Moderate_income_volume,LaborForce,Income_per_capita,Population
Date,,,,,,,,
2013-01-01,78702,1357,1392.0,5.560889e+08,2.954124e+09,1005093.0,47233.0,1883315.0
2013-01-01,78746,1282,1354.0,5.560889e+08,2.954124e+09,1005093.0,47233.0,1883315.0
2013-01-01,78703,1288,1481.0,5.560889e+08,2.954124e+09,1005093.0,47233.0,1883315.0
2013-01-01,78750,1325,1028.0,5.560889e+08,2.954124e+09,1005093.0,47233.0,1883315.0
2013-01-01,78757,1380,1101.0,5.560889e+08,2.954124e+09,1005093.0,47233.0,1883315.0
...,...,...,...,...,...,...,...,...
2018-07-01,78744,752,1283.0,8.639582e+08,4.220966e+09,1203826.0,59638.0,2165497.0
2018-07-01,78723,888,1386.0,8.639582e+08,4.220966e+09,1203826.0,59638.0,2165497.0
2018-07-01,78749,919,1221.0,8.639582e+08,4.220966e+09,1203826.0,59638.0,2165497.0


In [278]:
len(austin_df['Zipcode'].value_counts())

27

In [279]:
austin_df.isnull().sum(axis = 0)

Zipcode                   0
SizeRank                  0
Rent                      0
Low_income_volume         0
Moderate_income_volume    0
LaborForce                0
Income_per_capita         0
Population                0
dtype: int64

In [168]:
austin_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1809 entries, 2013-01-01 to 2018-07-01
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Zipcode                 1809 non-null   object 
 1   SizeRank                1809 non-null   int64  
 2   Rent                    1809 non-null   float64
 3   Low_income_volume       1809 non-null   float64
 4   Moderate_income_volume  1809 non-null   float64
 5   LaborForce              1809 non-null   float64
 6   NumEmp                  1809 non-null   float64
 7   Income_per_capita       1809 non-null   float64
 8   Population              1809 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 141.3+ KB


# Predict using random forest

In [280]:
aus_forest = austin_df.copy()
aus_forest['Zipcode'] = aus_forest['Zipcode'].astype(int)

from sklearn.ensemble import RandomForestRegressor

rfc = RandomForestRegressor()

In [281]:
from sklearn.model_selection import train_test_split
import numpy as np
y = np.log(aus_forest['Rent'])
X = aus_forest.drop('Rent', axis = 1)

Xtrain,Xtest,ytrain,ytest = train_test_split(X,y)

In [282]:
rfc.fit(Xtrain,ytrain)
rfc.score(Xtest,ytest)

0.9921029635269544

In [283]:
rfc.score(Xtrain,ytrain)

0.998777219884928

In [284]:
from sklearn.model_selection import cross_validate, KFold

cv = KFold(n_splits = 5, shuffle = True)

In [285]:
cv_res = cross_validate(RandomForestRegressor(),X,y, cv = cv)

In [286]:
cv_res2 = cross_validate(RandomForestRegressor(),Xtrain,ytrain,cv = cv)

In [287]:
for col, imp in zip(X.columns,rfc.feature_importances_):
    print(f'{col}: {imp}')

Zipcode: 0.6412048204519878
SizeRank: 0.17230421958148964
Low_income_volume: 0.025638453938848824
Moderate_income_volume: 0.026442764318582244
LaborForce: 0.12356267246022395
Income_per_capita: 0.004635747330603296
Population: 0.006211321918264237


In [255]:
len(aus_forest['Zipcode'].value_counts())

27

In [256]:
zipcodes.sample()

,geo_id,total_pop
20759,4444,8946


# Try random forest using population from BigQuery

In [257]:
zipcodes = pd.read_csv('~/Downloads/zipcode_2.csv')

In [258]:
zipcodes = zipcodes[['geo_id','total_pop']]

aus_zip = zipcodes.loc[zipcodes['geo_id'].isin(aus_forest['Zipcode'])]

In [259]:
aus_zip

,geo_id,total_pop
3005,78751,15805
4320,78727,29509
5735,78701,9427
9390,78705,33948
9416,78752,21324
9617,78729,29315
10243,78744,48969
11172,78757,24823
15172,78731,27175
15181,78730,9186


In [270]:
aus_forest2 = pd.merge(aus_forest,aus_zip, left_on = 'Zipcode', right_on = 'geo_id')

In [261]:
aus_forest2.drop(['Population','geo_id'], axis = 1, inplace = True)

In [271]:
np.sum(aus_zip['total_pop'])

883523

In [273]:
aus_forest2['Population'].value_counts()

2002134.0    324
1883315.0    324
2062016.0    324
2115475.0    324
1942467.0    324
2165497.0    189
Name: Population, dtype: int64

In [262]:
y = np.log(aus_forest2['Rent'])
X = aus_forest2.drop('Rent', axis = 1)

Xtrain,Xtest,ytrain,ytest = train_test_split(X,y)

In [263]:
rfc = RandomForestRegressor()
rfc.fit(Xtrain,ytrain)
rfc.score(Xtest,ytest)

0.9922435214917491

In [264]:
rfc.score(Xtrain,ytrain)

0.9989321758284088

In [265]:
from sklearn.model_selection import cross_validate, KFold

cv = KFold(n_splits = 5, shuffle = True)

In [266]:
cv_res = cross_validate(RandomForestRegressor(),X,y, cv = cv)

In [267]:
cv_res2 = cross_validate(RandomForestRegressor(),Xtrain,ytrain,cv = cv)

In [268]:
for col, imp in zip(X.columns,rfc.feature_importances_):
    print(f'{col}: {imp}')

Zipcode: 0.5831951395527925
SizeRank: 0.10910395531283357
Low_income_volume: 0.014231952871825115
Moderate_income_volume: 0.024333255086773045
LaborForce: 0.10044627293835216
NumEmp: 0.044125247755545625
Income_per_capita: 0.004924093038672145
total_pop: 0.11964008344320598
